# Домашнее задание 1. Сравнение методов коллаборативной фильтрации по сходству пользователей и по сходству объектов


Полезная ссылка на довольно подробный обзор по теме: http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf


1. Загрузите данные [Movie Lens](http://grouplens.org/datasets/movielens/), датасет на 1М рейтингов. Дополнительный бонус, если ваш код сможет обработать датасет на 10М. Я рекомендую сначала пробовать ваши методы на датасете из 100к рейтингов.  

2. Разбейте датасет на обучающую и тестовую выборку. 

Вариант 1: Тестовые пользователи -- это те, у кого ID делется на 5 без остатка.
Для каждого тестового пользователя скройте каждый десятый фильм по возрастанию ID.

Вариант 2: Тестовые пользователи -- это случайно выбранные 20% сохраните их список для воспроизводимости.
Для каждого тестового пользователя скройте каждый фильм с вероятностью 1/10, сохраните список скрытых фильмов.

Все пользователи и не скрытые фильмы это данные, на которых вы будете оценивать похожесть.

3. Выберите любые две меры похожести и реализуйте на их основе две модели рекомендаций на основе пользователей.
Для этих моделей посчитайте MAE и RMSE на всех скрытых оценках варианта 1. Какая модель оказалась лучше?

4. Повторите пункт 3 для варианта 2. Изменилось ли что-нибудь?

5. Для каждого тестового пользователя сгенерируйте рекомендации топ 10, 15, 20, 50, 100 фильмов у которых нет оценок. Посчитайте NDCG считая хорошими фильмы с оценкой 4 и выше.

6. Реализуте модель основанную на похожести предметов.

7. Повторите пункты 3-5 для этой модели. 

8. Насколько различны списки из top-n рекомендаций. Попробуйте улучшить результаты подбором $\beta$ для минимизации MAE (RMSE) в гибридных рекомендациях:
$$\beta\cdot r^{user-based}_{ui} + (1-\beta)\cdot r^{item-based}_{ui}, \mbox{ где } 0 \leq \beta \leq 1.$$ 

deadline 24.10.2021 23:59

In [39]:
import numpy as np
import pandas as pd
from math import sqrt
from numpy import exp
from random import random, seed

# Load date

In [1]:
#Чтение данных
def loadMovieLens1M(path='./ml-1m'):
    # Получить названия фильмов
    movies={}
    for line in open(path+'/movies.dat'):
        (id,title)=line.split('::')[0:2] #при желании тут можно добавить чтение жанров.
        movies[id]=title
    # Загрузить данные
    prefs={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [21]:
critics = loadMovieLens1M()
pd.DataFrame.from_dict(critics).head()

,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
One Flew Over the Cuckoo's Nest (1975),5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,5.0,5.0,4.0,NaN,NaN,4.0
James and the Giant Peach (1996),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
My Fair Lady (1964),3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN
Erin Brockovich (2000),4.0,NaN,NaN,NaN,3.0,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
"Bug's Life, A (1998)",5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN


## test train split

### Вариант 1:
Тестовые пользователи – это те, у кого ID делется на 5 без остатка. Для каждого тестового пользователя скройте каждый десятый фильм по возрастанию ID

In [16]:
all_users = [u for u in critics]
len(all_users)

6040

In [22]:
users_test_list = [u for u in critics if int(u) % 5 == 0]
len(users_test_list)

1208

In [34]:
critics_test = {}
# critics_train = {}
for u in users_test_list:
    critics_test[u] = {}
    # critics_train[u] = {}
    m_id = 0
    for m in critics[u]:
        m_id += 1
        if m_id % 10 == 0:
            critics_test[u][m] = critics[u][m]
        # else:
        #     critics_train[u][m] = critics[u][m]
pd.DataFrame.from_dict(critics_test).head()

,5,10,15,20,25,30,35,40,45,50,...,5995,6000,6005,6010,6015,6020,6025,6030,6035,6040
Donnie Brasco (1997),3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gladiator (2000),2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seven (Se7en) (1995),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apollo 13 (1995),2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Usual Suspects, The (1995)",5.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
len(critics_train)

6040

In [36]:
critics_train = {}
for u in all_users:
    if u not in users_test_list:
        critics_train[u] = {}
        for m in critics[u]:
            critics_train[u][m] = critics[u][m]
    else:
        critics_train[u] = {}
        m_id = 0
        for m in critics[u]:
            m_id += 1
            if m_id % 10 == 0:
                critics_train[u][m] = critics[u][m]

pd.DataFrame.from_dict(critics_train).head()

,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
One Flew Over the Cuckoo's Nest (1975),5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,NaN,5.0,4.0,NaN,NaN,NaN
James and the Giant Peach (1996),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
My Fair Lady (1964),3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN
Erin Brockovich (2000),4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
"Bug's Life, A (1998)",5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN


### Вариант 2:
Тестовые пользователи – это случайно выбранные 20% сохраните их список для воспроизводимости. Для каждого тестового пользователя скройте каждый фильм с вероятностью 1/10, сохраните список скрытых фильмов.

In [45]:
seed(2)
users_test_list_2 = [u for u in critics if random()<=0.2]
len(users_test_list_2)

1175

In [48]:
seed(2)
critics_test_2 = {}
critics_train_2_ = {}
for u in users_test_list_2:
    critics_test_2[u] = {}
    critics_train_2_[u] = {}
    for m in critics[u]:
        if random()<=0.2:
            critics_test_2[u][m] = critics[u][m]
        else:
            critics_train_2_[u][m] = critics[u][m]
pd.DataFrame.from_dict(critics_test_2).head()

,3,4,12,21,22,30,32,36,46,66,...,6000,6006,6008,6018,6021,6025,6028,6031,6032,6037
Mission: Impossible (1996),3.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Raising Arizona (1987),4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
Being John Malkovich (1999),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,4.0
American Beauty (1999),4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Toy Story 2 (1999),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
pd.DataFrame.from_dict(critics_train_2_).head()

,3,4,12,21,22,30,32,36,46,66,...,6000,6006,6008,6018,6021,6025,6028,6031,6032,6037
Animal House (1978),4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
"Full Monty, The (1997)",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28 Days (2000),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Happy Gilmore (1996),4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
"Golden Child, The (1986)",4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
critics_train_2 = {}
for u in all_users:
    if u not in users_test_list_2:
        critics_train_2[u] = {}
        for m in critics[u]:
            critics_train_2[u][m] = critics[u][m]
    else:
        critics_train_2[u] = {}
        for m in critics[u]:
            if m not in [m for m in critics_train_2_[u]]:
                critics_train_2[u][m] = critics[u][m]
pd.DataFrame.from_dict(critics_train_2).head()

,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
One Flew Over the Cuckoo's Nest (1975),5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,5.0,NaN,NaN,NaN,4.0
James and the Giant Peach (1996),3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
My Fair Lady (1964),3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN
Erin Brockovich (2000),4.0,NaN,NaN,NaN,3.0,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
"Bug's Life, A (1998)",5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN


Выберите любые две меры похожести и реализуйте на их основе две модели рекомендаций на основе пользователей. Для этих моделей посчитайте MAE и RMSE на всех скрытых оценках варианта 1. Какая модель оказалась лучше?